In [61]:
%matplotlib inline  
import matplotlib.pyplot as plt
from pamtra2.libs.singleScattering import scattering
from pamtra2.libs import refractiveIndex
import numpy as np
np.set_printoptions(precision=3,linewidth=100)

rad2deg = 180.0/np.pi

# One centimeter particle, S-band 2.8GHz, pure ice sphere 
f = 3.6e9
d = 0.001

t0 = np.pi*0.0 # theta must be within 0 pi
t1 = np.pi*0.5
#t1 = np.pi-t0
p0 = 1.1*np.pi
p1 = 1.4*np.pi
#p1 = np.pi+p0
print(p1, p1 // (2.*np.pi))
p1 = p1 - 2.*np.pi*(p1//(2.*np.pi))
print(t0, t1, p0, p1)

4.39822971502571 0.0
0.0 1.5707963267948966 3.455751918948773 4.39822971502571


In [62]:
n = refractiveIndex.n(frequencies=f, temperatures=263.15, substance='ice')
rayC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='Ray', theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
mieC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='Mie', theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
TmaC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='TMM', theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)

print(    'Cext           Csca         Cabs        Cbck[m**2]')
print(np.array(rayC[0:-1]))
print(np.array(mieC[0:-1]))
print(np.array(TmaC[0:-1]))
from pytmatrix import tmatrix, scatter, radar
scatt = tmatrix.Scatterer(radius=0.5*d, wavelength=299792458./f, m=n, axis_ratio=1.0/1.0, thet0=t0*rad2deg, thet=t1*rad2deg, phi0=p0*rad2deg, phi=p1*rad2deg)
print(np.array([scatter.ext_xsect(scatt),scatter.sca_xsect(scatt),scatter.ext_xsect(scatt)-scatter.sca_xsect(scatt),radar.radar_xsect(scatt)]))

Cext           Csca         Cabs        Cbck[m**2]
[  5.312e-12   7.511e-13   4.561e-12   1.127e-12]
[  5.318e-12   7.514e-13   4.567e-12   1.126e-12]
[  5.318e-12   7.514e-13   4.567e-12   1.126e-12]
[  5.318e-12   7.514e-13   4.567e-12   3.894e-13]


In [64]:
print(rayC[-1].flatten())
print(mieC[-1].flatten())
print(TmaC[-1].flatten())
print(scatt.get_S().flatten())

AttributeError: 'tuple' object has no attribute 'scatt_angle'

In [10]:
# One millimeter particle, S-band 2.8GHz, pure ice sphere 
f=2.8e9

n = refractiveIndex.n(frequencies=f,temperatures=263.15,substance='ice')
rayC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='Ray')
mieC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='Mie')
TmaC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='TMM')

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(rayC[0:-1])
print(mieC[0:-1])
print(TmaC[0:-1])

Cext                    Csca                    Cabs                   Cbck            [m**2]
(3.1485414853155008e-15, 2.7487221303700193e-19, 3.1482666131024637e-15, 4.1230831955550289e-19)
(3.1485646987497757e-15, 2.7487285968888451e-19, 3.1482898258900869e-15, 4.1230740899916799e-19)
(3.1485646291424952e-15, 2.7487285368539355e-19, 3.14828975628881e-15, 4.1230742079816574e-19)


In [8]:
scatt.get_S()
#scatt.get_Z()

array([[  2.99375135e-07 +3.18999523e-11j,
          5.98750267e-14 +6.37999044e-18j],
       [  5.98750267e-14 +6.37999045e-18j,
         -2.99375135e-07 -3.18999523e-11j]])

In [ ]:
import numpy as np
fig, ((ax00,ax01),(ax10,ax11)) = plt.subplots(2,2,figsize=(10,6),sharex=True)

sizes = np.linspace(0.0001,0.01,10) # from 0.1 mm to 1 cm
scaRay = 0.0*sizes
absRay = 0.0*sizes
bckRay = 0.0*sizes
extRay = 0.0*sizes
scaMie = 0.0*sizes
absMie = 0.0*sizes
bckMie = 0.0*sizes
extMie = 0.0*sizes
scaTMM = 0.0*sizes
absTMM = 0.0*sizes
bckTMM = 0.0*sizes
extTMM = 0.0*sizes

frequencies = {'S':2.8e9,'C':5.6e9,'Ku':13.6e9,'Ka':35.6e9,'W':94e9}
frequencies = {'S':2.8e9,'Ku':13.6e9,'W':94e9}
for fk in frequencies.keys():
    f = frequencies[fk]
    n = refractiveIndex.n(frequencies=f,temperatures=263.15,substance='ice')
    
    for i,s in enumerate(sizes):
        rayC = scattering(diameters=s, frequencies=f, refractive_indices=n,model='Ray')
        mieC = scattering(diameters=s, frequencies=f, refractive_indices=n,model='Mie')
        TmaC = scattering(diameters=s, frequencies=f, refractive_indices=n,model='TMM', theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
        #print(s,i)
        extMie[i] = mieC[0]
        scaMie[i] = mieC[1]
        absMie[i] = mieC[2]
        bckMie[i] = mieC[3]
        extRay[i] = rayC[0]
        scaRay[i] = rayC[1]
        absRay[i] = rayC[2]
        bckRay[i] = rayC[3]
        extTMM[i] = TmaC[0]
        scaTMM[i] = TmaC[1]
        absTMM[i] = TmaC[2]
        bckTMM[i] = TmaC[3]
    ax00.plot(sizes,extMie,ls='-.',label=fk+' Mie')
    ax01.plot(sizes,scaMie,ls='-.',label=fk+' Mie')
    ax10.plot(sizes,absMie,ls='-.',label=fk+' Mie')
    ax11.plot(sizes,bckMie,ls='-.',label=fk+' Mie')
    ax00.plot(sizes,extRay,label=fk+' Ray')
    ax01.plot(sizes,scaRay,label=fk+' Ray')
    ax10.plot(sizes,absRay,label=fk+' Ray')
    ax11.plot(sizes,bckRay,label=fk+' Ray')
    ax00.plot(sizes,extTMM,ls='--',label=fk+' TMM')
    ax01.plot(sizes,scaTMM,ls='--',label=fk+' TMM')
    ax10.plot(sizes,absTMM,ls='--',label=fk+' TMM')
    ax11.plot(sizes,bckTMM,ls='--',label=fk+' TMM')

ax00.set_yscale('log')
ax01.set_yscale('log')
ax10.set_yscale('log')
ax11.set_yscale('log')
ax00.set_xscale('log')
ax01.set_xscale('log')
ax10.set_xscale('log')
ax11.set_xscale('log')
ax11.legend()
ax00.grid()
ax01.grid()
ax10.grid()
ax11.grid()
ax00.set_ylabel('C$_{ext}$    [m$^2$]')
ax01.set_ylabel('C$_{sca}$    [m$^2$]')
ax10.set_ylabel('C$_{abs}$    [m$^2$]')
ax11.set_ylabel('C$_{bck}$    [m$^2$]')
ax10.set_xlabel('size    [m]')
ax11.set_xlabel('size    [m]')
ax00.set_title('Extinction')
ax01.set_title('Scattering')
ax10.set_title('Absorption')
ax11.set_title('Radar backscattering')
fig.tight_layout()

In [ ]:
mieC

In [ ]:
mieC[0]

#### Old code to test inner functionalities

In [ ]:
import numpy as np


import sys
sys.path.append('../') # singleScattering is not in my default path

from singleScattering import scatterer
from singleScattering import Rayleigh

%matplotlib inline

In [ ]:
try:
    generic = scatterer.scatterer()
except:
    pass
try:
    ray = Rayleigh.Rayleigh()
except:
    pass
try:
    TM = scatterer.T_Matrix()
except:
    pass
try:
    Mie = scatterer.Mie()
except:
    pass
#ray.back_spec

In [ ]:
ray = Rayleigh.RayleighScatt(frequency=1.0e9,dielectric_permittivity=complex(6.0,1.0))
# ray.set_scattering_geometry([0.5*np.pi, 0.0, 0.5*np.pi, 0.0])
# ray.set_scattering_geometry([0.0, 0.0, np.pi, 0.0])
# ray.set_scattering_geometry([np.pi, 0.0, np.pi, 0.0])

In [ ]:
ray.set_scattering_geometry([0.5*np.pi, 0.0, 0.5*np.pi, np.pi, 0.5*np.pi, np.pi])



In [ ]:
print(ray.wavelength,ray.Cbck)

In [ ]:
8.0*ray.size_parameter**4*ray.K2*ray.geometric_cross_section/3.0

In [ ]:
ray.Cabs

In [ ]:
print(ray.theta_inc,ray.theta_sca,ray.phi_inc,ray.phi_sca)
ray.scatt_angle
